In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!cp /gdrive/MyDrive/tmp/sign.zip ./

In [ ]:
!ls -al

total 2314788
drwxr-xr-x 1 root root       4096 Jul 31 01:02 .
drwxr-xr-x 1 root root       4096 Jul 31 01:02 ..
drwxr-xr-x 4 root root       4096 Jul 28 13:39 .config
drwxr-xr-x 1 root root       4096 Jul 28 13:40 sample_data
-rw------- 1 root root 2370319558 Jul 31 01:03 sign.zip


In [ ]:
!unzip sign.zip

Archive:  sign.zip
   creating: sign/
  inflating: sign/.DS_Store          
  inflating: __MACOSX/sign/._.DS_Store  
   creating: sign/test/
   creating: sign/train/
  inflating: sign/test/.DS_Store     
  inflating: __MACOSX/sign/test/._.DS_Store  
   creating: sign/test/10-1/
  inflating: __MACOSX/sign/test/._10-1  
   creating: sign/test/9/
  inflating: __MACOSX/sign/test/._9  
   creating: sign/test/7/
  inflating: __MACOSX/sign/test/._7  
   creating: sign/test/6/
  inflating: __MACOSX/sign/test/._6  
   creating: sign/test/1/
  inflating: __MACOSX/sign/test/._1  
   creating: sign/test/8/
  inflating: __MACOSX/sign/test/._8  
   creating: sign/test/10-2/
  inflating: __MACOSX/sign/test/._10-2  
   creating: sign/test/4/
  inflating: __MACOSX/sign/test/._4  
   creating: sign/test/3/
  inflating: __MACOSX/sign/test/._3  
   creating: sign/test/2/
  inflating: __MACOSX/sign/test/._2  
   creating: sign/test/5/
  inflating: __MACOSX/sign/test/._5  
  inflating: sign/train/.DS_Store 

In [ ]:
from IPython.display import clear_output
from tensorflow.keras.callbacks import Callback

#callback을 위한 클래스
class PlotLosses(Callback):

  def on_train_begin(self, logs={}):
    self.epochs = []
    self.losses = []
    self.val_losses = []
    self.logs = []
    self.fig = plt.figure()


  def on_epoch_end(self, epoch, logs={}):

    self.epochs.append(epoch)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))

    clear_output(wait=True)
    plt.plot(self.epochs, self.losses, label="loss")
    plt.plot(self.epochs, self.val_losses, label="val_loss")
    plt.legend()
    plt.show();
    print("loss = ", self.losses[-1], ", val_loss = ", self.val_losses[-1])

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB0

preprocessor = efficientnet.preprocess_input

#EfficientNetB0를 이용한 전이학습
conv_layers = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  
conv_layers.trainable = False

model = keras.Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())    #overffiting 처리
model.add(Dropout(0.2))            #overffiting 처리
model.add(Dense(11, activation='softmax'))  #1,2,3,4,5,6,7,8,9,10-1,10-2 총 11개의 클래스

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])  
model.summary()

BATCH_SIZE = 64

#augmentation(데이터 증강)
# train_data_generator = ImageDataGenerator(
#       rotation_range=10,
#       width_shift_range=0.1,
#       height_shift_range=0.1,
#       horizontal_flip=False,   #좌우반전 사용X 
#       vertical_flip=False,     #상하반전 사용X
#       preprocessing_function=preprocessor
# ).flow_from_directory(
#       "sign/train",
#       target_size=(224,224),
#       batch_size=BATCH_SIZE,
#       class_mode='sparse',
# )

train_data_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=False,   #좌우반전 사용X 
      vertical_flip=False,     #상하반전 사용X
      preprocessing_function=preprocessor,
      validation_split=0.1) 

train_generator = train_data_generator.flow_from_directory(
      "sign/train",
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse',
      subset='training'
) #train set

validation_generator = train_data_generator.flow_from_directory(
      "sign/train",
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse',
      subset='validation'
) #validtaion set

test_data_generator = ImageDataGenerator(
      preprocessing_function=preprocessor
).flow_from_directory(
      "sign/test",
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
) #test set



#loss가 최하인 best_model 저장
from tensorflow.keras.callbacks import ModelCheckpoint
model_check_point = ModelCheckpoint(
    'best_model.h5', 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True)

#20epochs 이상 loss가 개선되지 않으면 중지
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링 대상
    verbose=1,
    patience=30)         # 중지까지의 여유분 = 30epochs



#모델 학습, callback사용
history = model.fit(
      train_generator,
      validation_data=validation_generator,
      epochs=50,
      callbacks=[model_check_point, early_stopping]
)


#학습이 끝난 후 보여줄 plot들
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

loss, acc = model.evaluate(test_data_generator)
print("loss=", loss)
print("acc=", acc)

batch_x, batch_y = test_data_generator.next()
y_ = model.predict(batch_x)
predicted = np.argmax(y_, axis=-1)

plt.plot(batch_y[:100], "o")
plt.plot(predicted[:100], '.')
plt.show()

custom_labels = list(test_data_generator.class_indices.keys())
print("label category index =", batch_y[0])
print("predicted category index =", predicted[0])
print("predicted category name =", custom_labels[predicted[0]])



16719872/16705208 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 256)               16056576  
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 11)                28

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-e34d96078c73>", line 114, in <module>
    callbacks=[model_check_point, early_stopping]
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1431, in fit
    _use_cached_eval_dataset=True)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1716, in evaluate
    tmp_logs = self.test_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  

KeyboardInterrupt: ignored